In [23]:
import pandas as pd

# Load the dataset
geo_device = pd.read_csv("input_data/estat_isoc_ci_dev_i_en.csv")
geo_beds = pd.read_csv("input_data/estat_tour_cap_nat_en.csv")

In [24]:
#check the data
print(geo_device.head())

                   DATAFLOW        LAST UPDATE freq   ind_type    indic_is  \
0  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
1  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
2  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
3  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
4  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   

     unit geo  TIME_PERIOD  OBS_VALUE OBS_FLAG  CONF_STATUS  
0  PC_IND  AL         2018        NaN        u          NaN  
1  PC_IND  AT         2016      37.41      NaN          NaN  
2  PC_IND  AT         2018      42.45      NaN          NaN  
3  PC_IND  AT         2021      39.62      NaN          NaN  
4  PC_IND  AT         2023      40.34      NaN          NaN  


In [12]:
print(geo_beds.columns)
print(geo_device.columns)

Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'accomunit', 'unit', 'nace_r2',
       'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG', 'CONF_STATUS'],
      dtype='object')
Index(['DATAFLOW', 'LAST UPDATE', 'freq', 'ind_type', 'indic_is', 'unit',
       'geo', 'TIME_PERIOD', 'OBS_VALUE', 'OBS_FLAG', 'CONF_STATUS'],
      dtype='object')


In [ ]:
# cols to keep
geo_beds = geo_beds[['geo','TIME_PERIOD','accomunit', 'unit', 'nace_r2']]
print(geo_beds.head())

  geo  TIME_PERIOD accomunit unit nace_r2
0  AT         1990     BEDPL   NR    I551
1  AT         1991     BEDPL   NR    I551
2  AT         1992     BEDPL   NR    I551
3  AT         1993     BEDPL   NR    I551
4  AT         1994     BEDPL   NR    I551


In [21]:
# geo_beds['geo'].unique()
# geo_beds['nace_r2'].unique()
# geo_beds['accomunit'].unique()
# geo_beds['unit'].unique()
geo_beds['TIME_PERIOD'].unique()

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022,
       2023, 2024])

In [ ]:
# Rows to use: You should only use rows where: 
# “accomunit” is “BEDPL”, --> bedplaces
# “unit” is “NR”, --> number
# “nace_r2” is “I551”, --> hotels and similar accommodation 
# “TIME_PERDIO” is “2016” 

# Output the number of bedplaces in hotels and similar accommodation in 2016
geo_beds_2016 = geo_beds[(geo_beds['accomunit'] == 'BEDPL') & (geo_beds['unit'] == 'NR') & (geo_beds['nace_r2'] == 'I551') & (geo_beds['TIME_PERIOD'] == '2016')]

print(geo_beds_2016.head())

In [5]:
#check the data
print(geo_device.head())

                   DATAFLOW        LAST UPDATE freq   ind_type    indic_is  \
0  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
1  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
2  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
3  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   
4  ESTAT:ISOC_CI_DEV_I(1.0)  17/12/24 11:00:00    A  CB_EU_FOR  I_IUG_DKPC   

     unit geo  TIME_PERIOD  OBS_VALUE OBS_FLAG  CONF_STATUS  
0  PC_IND  AL         2018        NaN        u          NaN  
1  PC_IND  AT         2016      37.41      NaN          NaN  
2  PC_IND  AT         2018      42.45      NaN          NaN  
3  PC_IND  AT         2021      39.62      NaN          NaN  
4  PC_IND  AT         2023      40.34      NaN          NaN  
